# Step 3: Training a LoRA Adapter

This notebook performs the preparatory tasks needed for obtaining the base model that we will use for fine-tuning.

This notebook showcases performing LoRA fine-tuning on the dataset that we curated in step 1.

## Setup and Requirements
Before proceeding, please make ensure you have completed the notebooks for steps 1 and 2. You will need to install one dependency to follow along. Execute the following cell before getting started.

In [2]:
# ! pip install ipywidgets

Let's also specify the base model name that we will use for fine-tuning. This should be the same model you downloaded/converted in step 2.

In [1]:
model_to_use = "google/gemma-2-2b"

---
# Sanity Checking

Let's do a quick sanity check to ensure we have all the pieces needed before moving forward.

In [2]:
import os

model_name = model_to_use.split('/')[-1].lower()

# The path to the model checkpoint, and also the data directory containing the training, validation, and test data.
nemo_model_fp = os.path.abspath(f"models/{model_name}.nemo")
data_dir = "data/split"

# The directory where the results will be stored.
result_dir = os.path.abspath("results")
os.makedirs(result_dir, exist_ok=True)

# Sanity checks
assert os.path.exists(nemo_model_fp), f"The model checkpoint at '{nemo_model_fp}' does not exist. Please ensure the model was downloaded successfully."
assert os.path.exists(data_dir), f"The data directory '{data_dir}' does not exist. Please ensure the data was prepared successfully."

train_fp = os.path.abspath(f"{data_dir}/train.jsonl")
val_fp = os.path.abspath(f"{data_dir}/val.jsonl")

# Sanity checks
assert os.path.exists(train_fp), f"The training data at '{train_fp}' does not exist. Please ensure the data was prepared successfully."
assert os.path.exists(val_fp), f"The validation data at '{val_fp}' does not exist. Please ensure the data was prepared successfully."

#
# Set the environment variables (needed for executing the next cell)
#
%env BASE_MODEL=$nemo_model_fp
%env DATA_DIR=$data_dir
%env TRAIN_DS=$train_fp
%env VAL_DS=$val_fp
%env RESULT_DIR=$result_dir

print(f"\n{'#'*80}")
print("All checks passed. You are ready to go!")
print(f"    Base model file: {nemo_model_fp}")
print(f"    Data directory: {data_dir}")
print(f"    Results: {result_dir}")

env: BASE_MODEL=/root/ODSC-Hackathon-Repository/models/gemma-2-2b.nemo
env: DATA_DIR=data/split
env: TRAIN_DS=/root/ODSC-Hackathon-Repository/data/split/train.jsonl
env: VAL_DS=/root/ODSC-Hackathon-Repository/data/split/val.jsonl
env: RESULT_DIR=/root/ODSC-Hackathon-Repository/results

################################################################################
All checks passed. You are ready to go!
    Base model file: /root/ODSC-Hackathon-Repository/models/gemma-2-2b.nemo
    Data directory: data/split
    Results: /root/ODSC-Hackathon-Repository/results


---
# Model Training

With all the sanity checks passing, it is time to start model training.

> NOTE: Running the following cell will remove any previously trained model!

* Our choice of bf16 precision strikes a balance between training speed and memory usage, crucial for handling our large language model. 



* The micro batch size of 1 and global batch size of 16 were chosen to optimize memory usage and gradient accumulation, allowing for stable training.



* We implemented a cosine annealing learning rate schedule with a base rate of 1e-6. This approach helps in finding an optimal convergence point by gradually reducing the learning rate, preventing overshooting in later stages of training.



* Infact, during our first trial of fine tuning the model, the training and validation loss are too far apart and also the validation is not converging while the training loss is, which is a clear sight of overfitting.



* So, we introduced weight decay of 0.01 was selected to prevent overfitting and ensure smooth gradient updates.



* We set our max steps to 2500 with evaluations every 200 steps. This frequent evaluation allowed us to monitor the model's performance closely and make necessary adjustments.



* We have used optimizer beta values of 0.9 and 0.95 to help achieve faster convergence and improved stability during the fine-tuning of large language models by adjusting the decay rates of the gradient and variance estimates in the Adam optimizer.


In [5]:
%%bash

SCHEME="lora"
TP_SIZE=1
PP_SIZE=1

# Clear up cached mem-map file
rm $DATA_DIR/*idx*
# Clean up prior results
rm -r $RESULT_DIR

HYDRA_FULL_ERROR=1 torchrun --nproc_per_node=1 \
/opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
    exp_manager.exp_dir=${RESULT_DIR} \
    exp_manager.explicit_log_dir=${RESULT_DIR} \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    trainer.precision=bf16 \
    trainer.val_check_interval=200 \
    trainer.max_steps=2500 \
    trainer.gradient_clip_val=0.5 \
    model.megatron_amp_O2=True \
    ++model.mcore_gpt=True \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    model.micro_batch_size=1 \
    model.global_batch_size=16 \
    model.optim.sched.name="CosineAnnealing" \
    model.optim.sched.warmup_steps=200 \
    model.optim.lr=1e-6 \
    model.optim.weight_decay=0.01 \
    model.optim.betas=[0.9,0.95] \
    model.restore_from_path=${BASE_MODEL} \
    model.data.train_ds.num_workers=2 \
    model.data.train_ds.add_bos=True \
    model.data.validation_ds.num_workers=1 \
    model.data.train_ds.file_names=[${TRAIN_DS}] \
    model.data.train_ds.concat_sampling_probabilities=[1.0] \
    model.data.validation_ds.file_names=[${VAL_DS}] \
    model.peft.peft_scheme=${SCHEME}

rm: cannot remove 'data/split/*idx*': No such file or directory
[NeMo W 2024-10-28 01:10:40 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pyannote/core/notebook.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
      cm = get_cmap("Set1")
    


[NeMo I 2024-10-28 01:10:42 megatron_gpt_finetuning:56] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-10-28 01:10:42 megatron_gpt_finetuning:57] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: bf16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 2500
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 0.5
    exp_manager:
      explicit_log_dir: /root/ODSC-Hackathon-Repository/results
      exp_dir: /root/ODSC-Hackathon-Repository/results
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.validat

[NeMo W 2024-10-28 01:10:42 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[NeMo I 2024-10-28 01:10:42 exp_manager:450] ExpManager schema
[NeMo I 2024-10-28 01:10:42 exp_manager:451] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'log_

[NeMo E 2024-10-28 01:10:42 exp_manager:910] exp_manager received explicit_log_dir: /root/ODSC-Hackathon-Repository/results and at least one of exp_dir: /root/ODSC-Hackathon-Repository/results, or version: None. Please note that exp_dir, name, and version will be ignored.
[NeMo W 2024-10-28 01:10:42 exp_manager:837] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :/root/ODSC-Hackathon-Repository/results/checkpoints. Training from scratch.


...

`Trainer.fit` stopped: `max_steps=2500` reached.


Epoch 0: : 100%|██████████| 2500/2500 [2:44:54<00:00, reduced_train_loss=1.930, global_step=2499.0, consumed_samples=4e+4, train_step_timing in s=3.420, val_loss=2.460]   
[NeMo I 2024-10-28 03:56:43 perf_metrics:87] TFLOPs per sec per GPU=-1.00


[NeMo E 2024-10-28 03:56:43 perf_metrics:85] Failed to calculate TFLOPs per sec per GPU.
    FLOPs measurement not supported for finetuning jobs


[NeMo I 2024-10-28 03:56:44 nlp_overrides:609] Removing checkpoint: /root/ODSC-Hackathon-Repository/results/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=2.464-step=2400-consumed_samples=38400.0-last.ckpt


Restoring states from the checkpoint path at /root/ODSC-Hackathon-Repository/results/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=2.464-step=2400-consumed_samples=38400.0.ckpt
Restored all states from the checkpoint at /root/ODSC-Hackathon-Repository/results/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=2.464-step=2400-consumed_samples=38400.0.ckpt


---
# Inference and Submission


To make a submission, run inference with your model on the test dataset at `data/split/submission.jsonl`.

> NOTE: This dataset was generated as part of Step 1. Please ensure it exists before proceeding.

In order to do this, set the variable pointing to your submission data file in the set below, then excute the final cell.

The inference results will be written under `results/inference` folder.

In [3]:
test_fp = os.path.abspath(f"{data_dir}/submission.jsonl")
assert os.path.exists(test_fp), f"The submission data at '{test_fp}' does not exist. Please ensure the data was prepared successfully."

test_fp = os.path.abspath(test_fp)
adapter_fp = f"{result_dir}/checkpoints/megatron_gpt_peft_lora_tuning.nemo"
os.makedirs(f"{result_dir}/inference", exist_ok=True)

print(f"Inference set: {test_fp}")
print(f"Trained adapter: {adapter_fp}")
test_filename = os.path.basename(test_fp)


%env TEST_DS=$test_fp
%env TEST_FP=$test_filename
%env TRAINED_ADAPTER=$adapter_fp

Inference set: /root/ODSC-Hackathon-Repository/data/split/submission.jsonl
Trained adapter: /root/ODSC-Hackathon-Repository/results/checkpoints/megatron_gpt_peft_lora_tuning.nemo
env: TEST_DS=/root/ODSC-Hackathon-Repository/data/split/submission.jsonl
env: TEST_FP=submission.jsonl
env: TRAINED_ADAPTER=/root/ODSC-Hackathon-Repository/results/checkpoints/megatron_gpt_peft_lora_tuning.nemo


# Inference

In [4]:
%%bash

# This is where the inference results will be stored.
OUTPUT_DIR="results/inference/infer-$TEST_FP"

SCHEME="lora"
TP_SIZE=1
PP_SIZE=1

# Clear up cached mem-map file
rm $DATA_DIR/*idx*

python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
    model.restore_from_path=${BASE_MODEL} \
    model.peft.restore_from_path=${TRAINED_ADAPTER} \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    inference.greedy=True \
    model.data.test_ds.file_names=[${TEST_DS}] \
    model.data.test_ds.names=["infer"] \
    model.data.test_ds.global_batch_size=32 \
    model.data.test_ds.micro_batch_size=1 \
    model.data.test_ds.tokens_to_generate=32 \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    model.data.test_ds.output_file_path_prefix=$OUTPUT_DIR \
    model.data.test_ds.write_predictions_to_file=True

[NeMo W 2024-10-28 05:26:12 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pyannote/core/notebook.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
      cm = get_cmap("Set1")
    


[NeMo I 2024-10-28 05:26:13 megatron_gpt_generate:125] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-10-28 05:26:13 megatron_gpt_generate:126] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo_o

[NeMo W 2024-10-28 05:26:13 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it 

[NeMo I 2024-10-28 05:26:19 megatron_init:314] Rank 0 has data parallel group : [0]
[NeMo I 2024-10-28 05:26:19 megatron_init:320] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-10-28 05:26:19 megatron_init:325] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-10-28 05:26:19 megatron_init:328] Ranks 0 has data parallel rank: 0
[NeMo I 2024-10-28 05:26:19 megatron_init:336] Rank 0 has context parallel group: [0]
[NeMo I 2024-10-28 05:26:19 megatron_init:339] All context parallel group ranks: [[0]]
[NeMo I 2024-10-28 05:26:19 megatron_init:340] Ranks 0 has context parallel rank: 0
[NeMo I 2024-10-28 05:26:19 megatron_init:347] Rank 0 has model parallel group: [0]
[NeMo I 2024-10-28 05:26:19 megatron_init:348] All model parallel group ranks: [[0]]
[NeMo I 2024-10-28 05:26:19 megatron_init:357] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-10-28 05:26:19 megatron_init:361] All tensor model parallel group ranks: 

[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: finalize_model_grads_func in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-10-28 05:26:19 megatron_base_model:1189] The model: MegatronGPTSFTModel() does not have field.name: deterministi

...

[NeMo W 2024-10-28 05:26:38 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:161: You have overridden `MegatronGPTSFTModel.configure_sharded_model` which is deprecated. Please override the `configure_model` hook instead. Instantiation with the newer hook will be created on the device right away and have the right data type depending on the precision setting in the Trainer.
    
[NeMo W 2024-10-28 05:26:38 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:143: You are using the `dataloader_iter` step flavor. If you consume the iterator more than once per step, the `batch_idx` argument in any hook that takes it will not match with the batch index of the last batch consumed. This might have unforeseen effects on callbacks or code that expects to get the correct index. This will also not work well with gradient accumulation. This feature is very experimental and subjec

[NeMo I 2024-10-28 05:26:38 megatron_gpt_sft_model:828] Building GPT SFT test datasets.
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:116] Building data files
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:528] Processing 1 data files using 6 workers
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:494] Building indexing for fn = /root/ODSC-Hackathon-Repository/data/split/submission.jsonl
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:506] Saving idx file = /root/ODSC-Hackathon-Repository/data/split/submission.jsonl.idx.npy
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:508] Saving metadata file = /root/ODSC-Hackathon-Repository/data/split/submission.jsonl.idx.info
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:543] Time building 1 / 1 mem-mapped files: 0:00:00.214818
[NeMo I 2024-10-28 05:26:38 text_memmap_dataset:528] Processing 1 data files using 6 workers
[NeMo I 2024-10-28 05:26:39 text_memmap_dataset:543] Time building 0 / 1 mem-mapped files: 0:00:00.201747
[NeMo I 2024-10-2

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2024-10-28 05:26:39 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
    


Testing: |          | 0/? [00:00<?, ?it/s]setting number of microbatches to constant 32
Testing DataLoader 0:   0%|          | 0/157 [00:00<?, ?it/s]setting number of microbatches to constant 1
setting number of microbatches to constant 32
Testing DataLoader 0:   1%|          | 1/157 [00:19<50:16,  0.05it/s]setting number of microbatches to constant 1
setting number of microbatches to constant 32
Testing DataLoader 0:   1%|▏         | 2/157 [00:35<45:37,  0.06it/s]setting number of microbatches to constant 1
setting number of microbatches to constant 32
Testing DataLoader 0:   2%|▏         | 3/157 [00:49<42:42,  0.06it/s]setting number of microbatches to constant 1
setting number of microbatches to constant 32
Testing DataLoader 0:   3%|▎         | 4/157 [01:04<40:48,  0.06it/s]setting number of microbatches to constant 1
setting number of microbatches to constant 32
Testing DataLoader 0:   3%|▎         | 5/157 [01:19<40:31,  0.06it/s]setting number of microbatches to constant 1
settin

[NeMo W 2024-10-28 06:10:53 megatron_gpt_sft_model:677] No training data found, reconfiguring microbatches based on validation batch sizes.
[NeMo W 2024-10-28 06:10:53 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-10-28 06:10:53 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss_infer', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-10-28 06:10:53 nemo_logging:361] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss', ..., syn

The results will be written under `results/inference`. Please send us this file for your final submission.

Let's inspect a couple of lines from that file for sanity checking:

In [5]:
! cat results/inference/infer-submission.jsonl_test_infer_inputs_preds_labels.jsonl | head -n 1

{"input": "\nYou are an expert in Law and also in tagging legal questions.\nYou are provided with a question enclosed in +++++ and it's corresponding title enclosed in >>>>> from the law domain.\n\nYou are also provided with a list of all the tags, enclosed in ^^^^^.\n\nYour task is to:\ni. Understand the question and it's title.\nii. Pick up the tags that are most appropriate and relevant to the question, strictly from the tags provided to you.\niii. Make sure you return the tags alone without their description.\n\n\n```\nNOTES: All tags must be in lowercase, ordered lexicographically and separated by commas.\n```\n\nYour output should be a JSON with the below format:\n```\ntags : <Put your relevant tags here>\n```\n\n\n>>>>>\nTitle: Fairness in Punishment for Reckless Behavior\n>>>>>\n\n\n+++++\nQuestion: Is it justifiable to have significantly different penalties for individuals who engage in reckless behavior, depending on the outcome of their actions, or should the focus be on the

---
# Freeing Memory and Other Resources

As always, it is a good idea to free up all allocated resources when you are done. Please execute the following cell to do so.

Alternatively, please restart the kernel by navigating to `Kernel > Restart Kernel` (if using Jypyter notebook), or clicking the `Restart` button in VS Code.

In [ ]:
exit(0)